In [153]:
# Importing libraries:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Importing the os library specifically for reading the csv once I've created the file in my working directory.
import os

# web-based requests
import requests

In [154]:
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



In [155]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [ ]:
# Now I have the info I need to start getting specific portions of the information via the API.

## Items Data

### 1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [156]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [157]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [158]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 3
next_page: /api/v1/items?page=2


In [159]:
data['payload']['items'][:2]

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

In [142]:
# So I need to go back and recall how to work with dictionaries. I need to use those techniques to parse the info, instead of trying to use API to prep the data.

In [143]:
# data from page 1

items = pd.DataFrame(data['payload']['items'])
print(items.shape)
items.head()

(20, 6)


,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [144]:
data['payload']['items']

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [146]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: /api/v1/items?page=3


In [148]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: None


In [150]:
items.drop(columns = ['level_0', 'index'], inplace = True)
items.shape

(50, 6)

In [151]:
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [189]:
items_df = pd.DataFrame(data['payload']['items'])
print(items_df.shape)
items_df.head()

(20, 6)


,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [152]:
# I want to try creating an if loop for this one:

In [167]:
data['payload']['max_page']

3

In [184]:
response_test = requests.get('https://python.zach.lol/api/v1/items')

In [185]:
data_test = response.json()

In [186]:
response_test = requests.get(base_url + data_test['payload']['next_page'])
data_test = response_test.json()
print(data_test['payload']['next_page'])

/api/v1/items?page=3


In [187]:
response_test = requests.get(base_url + data_test['payload']['next_page'])
data_test = response_test.json()
print(data_test['payload']['next_page'])

None


In [190]:
data['payload']['max_page']

3

In [207]:
# Creating the function:

def get_items_new(base_url):
    '''
    This function is designed to get the items data from Zach's web service and turn that data into a pandas
    dataframe for use.
    '''
    
    # initialize:
    
    response = requests.get('https://python.zach.lol/api/v1/items')
    data = response.json()
    df = pd.DataFrame(data['payload']['items'])
    
    
    
    for x in range(0, data['payload']['max_page']):
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        df = pd.concat([df, pd.DataFrame(data['payload']['items'])], ignore_index = True)
        if data['payload']['next_page'] == None:
            return df
    df = df.reset_index()
    return df

In [208]:
base_url = 'https://python.zach.lol'

items_df_testing = get_items_new(base_url)

In [209]:
items.shape, items_df_testing.shape

((50, 6), (50, 6))

In [210]:
items_df_testing.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [206]:
base_url + '/api/v1/stores'

'https://python.zach.lol/api/v1/stores'

## Store Data

### 2. Do the same thing, but for stores.



In [211]:
response_stores = requests.get('https://python.zach.lol/api/v1/stores')

data_stores = response_stores.json()
data_stores.keys()

dict_keys(['payload', 'status'])

In [212]:
data_stores['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [213]:
print('max_page: %s' % data_stores['payload']['max_page'])
print('next_page: %s' % data_stores['payload']['next_page'])

max_page: 1
next_page: None


In [214]:
data_stores['payload'].keys()
data_stores['payload']['stores'][:2]

[{'store_address': '12125 Alamo Ranch Pkwy',
  'store_city': 'San Antonio',
  'store_id': 1,
  'store_state': 'TX',
  'store_zipcode': '78253'},
 {'store_address': '9255 FM 471 West',
  'store_city': 'San Antonio',
  'store_id': 2,
  'store_state': 'TX',
  'store_zipcode': '78251'}]

In [215]:
stores_df = pd.DataFrame(data_stores['payload']['stores'])
print(stores_df.shape)
stores_df

(10, 5)


,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


In [ ]:
# Only one page of store data, so I don't need to concatinate multiple pages here.

In [228]:
# Creating the function:

def get_stores_new(base_url):
    '''
    This function is designed to get the items data from Zach's web service and turn that data into a pandas
    dataframe for use.
    '''
    
    # initialize:
    
    response = requests.get('https://python.zach.lol/api/v1/stores')
    data = response.json()
    df = pd.DataFrame(data['payload']['stores'])
    
    
    if data['payload']['next_page'] == None:
        return df
    else:
        for x in range(0, data['payload']['max_page']):
            response = requests.get(base_url + data['payload']['next_page'])
            data = response.json()
            df = pd.concat([df, pd.DataFrame(data['payload']['stores'])], ignore_index = True)
        return df
    df = df.reset_index()
    return df

In [229]:
stores_df_test = get_stores_new(base_url)
stores_df.shape, stores_df_test.shape

((10, 5), (10, 5))

## Sales Data

### 3. Extract the data for sales. 

- There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [ ]:
base_url_sales = 'https://python.zach.lol'

In [ ]:
# Create a function from what I'm doing here. While loop is based on next_page != "None"

response_sales = requests.get('https://python.zach.lol/api/v1/sales')

data_sales = response_sales.json()
data_sales.keys()

In [ ]:
data_sales['payload'].keys()

In [ ]:
print('max_page: %s' % data_sales['payload']['max_page'])
print('next_page: %s' % data_sales['payload']['next_page'])

In [ ]:
# This code is looking at a set number of entries in the dictionary I'm calling from the api:
data_sales['payload']['sales'][:2]

In [ ]:
sales_df = pd.DataFrame(data_sales['payload']['sales'])
sales_df.head()

In [ ]:
sales_df.shape

In [ ]:
response_sales = requests.get(base_url + data_sales['payload']['next_page'])
data_sales = response_sales.json()

print('max_page: %s' % data_sales['payload']['max_page'])
print('next_page: %s' % data_sales['payload']['next_page'])

sales_df = pd.concat([sales_df, pd.DataFrame(data_sales['payload']['sales'])]).reset_index()
sales_df.shape

In [ ]:
# Calling the same thing, but now I'm calling as many items are on the first page:
len(data_sales['payload']['sales'])
# There are 5000 items per page it seems. So the number of rows I'll have is:

row_total_guess = len(data_sales['payload']['sales']) * (data_sales['payload']['max_page'])
print(f'The estimated total number of rows of the combined sales dataframe is {row_total_guess:,}.')

In [231]:
def get_sales(base_url):
    
    response = requests.get('https://python.zach.lol/api/v1/sales')
    data = response.json()
    data.keys()
    print('max_page: %s' % data['payload']['max_page'])
    print('next_page: %s' % data['payload']['next_page'])
    
    df_sales = pd.DataFrame(data['payload']['sales'])
    
    while data['payload']['next_page'] != "None":
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        print('max_page: %s' % data['payload']['max_page'])
        print('next_page: %s' % data['payload']['next_page'])
        
        
        df_sales = pd.concat([df_sales, pd.DataFrame(data['payload']['sales'])])
        
        if data['payload']['next_page'] == None:
            break
            
    df_sales = df_sales.reset_index()
    print('full_shape', df_sales.shape)
    return df_sales
    
    

In [232]:
sales_df = get_sales(base_url_sales)

max_page: 183
next_page: /api/v1/sales?page=2
max_page: 183
next_page: /api/v1/sales?page=3
max_page: 183
next_page: /api/v1/sales?page=4
max_page: 183
next_page: /api/v1/sales?page=5
max_page: 183
next_page: /api/v1/sales?page=6
max_page: 183
next_page: /api/v1/sales?page=7
max_page: 183
next_page: /api/v1/sales?page=8
max_page: 183
next_page: /api/v1/sales?page=9
max_page: 183
next_page: /api/v1/sales?page=10
max_page: 183
next_page: /api/v1/sales?page=11
max_page: 183
next_page: /api/v1/sales?page=12
max_page: 183
next_page: /api/v1/sales?page=13
max_page: 183
next_page: /api/v1/sales?page=14
max_page: 183
next_page: /api/v1/sales?page=15
max_page: 183
next_page: /api/v1/sales?page=16
max_page: 183
next_page: /api/v1/sales?page=17
max_page: 183
next_page: /api/v1/sales?page=18
max_page: 183
next_page: /api/v1/sales?page=19
max_page: 183
next_page: /api/v1/sales?page=20
max_page: 183
next_page: /api/v1/sales?page=21
max_page: 183
next_page: /api/v1/sales?page=22
max_page: 183
next_pa

max_page: 183
next_page: /api/v1/sales?page=175
max_page: 183
next_page: /api/v1/sales?page=176
max_page: 183
next_page: /api/v1/sales?page=177
max_page: 183
next_page: /api/v1/sales?page=178
max_page: 183
next_page: /api/v1/sales?page=179
max_page: 183
next_page: /api/v1/sales?page=180
max_page: 183
next_page: /api/v1/sales?page=181
max_page: 183
next_page: /api/v1/sales?page=182
max_page: 183
next_page: /api/v1/sales?page=183
max_page: 183
next_page: None
full_shape (913000, 6)


In [233]:
sales_df.shape

(913000, 6)

In [235]:
sales_df.drop(columns = 'index', inplace = True)

In [236]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [160]:
# Writing to a csv:

def write_csv(df, csv_name):
    '''
    The first argument (df) is the dataframe you want written to a .csv file. 
    The second argument (csv_name) must be a string, including the .csv extention. eg: 'example_df.csv'
    '''
    
    df.to_csv(csv_name, index = False)
    print('Completed writing df to .csv file')
    

In [85]:
write_csv(sales_df, 'sales_df.csv')

Completed writing df to .csv file


In [86]:
write_csv(stores_df, 'stores_df.csv')

Completed writing df to .csv file


In [87]:
write_csv(items, 'items_df.csv')

Completed writing df to .csv file


### 5. Combine the data from your three separate dataframes into one large dataframe.

In [88]:
print(sales_df.shape)
sales_df.head()

(913000, 5)


,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [89]:
print(stores_df.shape)
stores_df.head()

(10, 5)


,store_address,store_city,store,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [90]:
print(items.shape)
items.head()

(20, 6)


,item_brand,item,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [91]:
# so, sales_df has both of the ids needed to join the two tables. I'll need to use two joins, both of them (I believe) will be a left join.

sales_test = sales_df.copy()

In [92]:
item_test = items.copy()
item_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  20 non-null     object 
 1   item        20 non-null     int64  
 2   item_name   20 non-null     object 
 3   item_price  20 non-null     float64
 4   item_upc12  20 non-null     object 
 5   item_upc14  20 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ KB


In [113]:
items.rename(columns = {'item_id': 'item'}, inplace = True)
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  20 non-null     object 
 1   item        20 non-null     int64  
 2   item_name   20 non-null     object 
 3   item_price  20 non-null     float64
 4   item_upc12  20 non-null     object 
 5   item_upc14  20 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ KB


In [94]:
stores_df.rename(columns = {'store_id': 'store'}, inplace = True)
stores_df.head()

,store_address,store_city,store,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [114]:
items.head()

,item_brand,item,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [115]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [117]:
left_merge = pd.merge(sales_df, items, how = 'left', on = 'item')
left_merge.head()

,item,sale_amount,sale_date,sale_id,store,item_brand,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [118]:
left_merge.shape

(913000, 10)

In [119]:
all_df = pd.merge(left_merge, stores_df, how = 'left', on = 'store')
all_df.head()

,item,sale_amount,sale_date,sale_id,store,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [120]:
all_df.shape

(913000, 14)

In [121]:
# Saving to csv:

write_csv(all_df, 'add_df.csv')

Completed writing df to .csv file


## Open Power System Data

### Acquire the Open Power Systems Data for Germany.

- Which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. 

In [126]:
power_url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
power_df = pd.read_csv(power_url, ',')
power_df.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [132]:
# A little bit of prep work:

power_df.rename(columns = {"Date": 'date', "Consumption": "consumption", "Wind": "wind", "Solar": "solar", "Wind+Solar": "wind_solar"}, inplace = True)
power_df.columns

Index(['date', 'consumption', 'wind', 'solar', 'wind_solar'], dtype='object')

In [136]:
# Now the function:

def get_germany_power(power_url):
    df = pd.DataFrame()
    df = pd.read_csv(power_url, ',')
    
    # now the cleaning:
    df.rename(columns = {"Date": 'date', "Consumption": "consumption", "Wind": "wind", "Solar": "solar", "Wind+Solar": "wind_solar"}, inplace = True)
    
    return df    

In [137]:
# testing the function..

power_df_test = get_germany_power(power_url)
power_df_test.head()

,date,consumption,wind,solar,wind_solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [238]:
# writing to a csv:

write_csv(power_df_test, 'power_df.csv')

Completed writing df to .csv file


## Functions for Reproducibility

### 7. Make sure all the work that you have done above is reproducible. 
- That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [ ]:
# Creating the items function:

def get_items_new(base_url):
    '''
    This function is designed to get the items data from Zach's web service and turn that data into a pandas
    dataframe for use.
    '''
    
    # initialize:
    
    response = requests.get('https://python.zach.lol/api/v1/items')
    data = response.json()
    df = pd.DataFrame(data['payload']['items'])
    
    
    
    for x in range(0, data['payload']['max_page']):
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        df = pd.concat([df, pd.DataFrame(data['payload']['items'])], ignore_index = True)
        if data['payload']['next_page'] == None:
            return df
    df = df.reset_index()
    return df

In [237]:
# stores function:

def get_stores_new(base_url):
    '''
    This function is designed to get the items data from Zach's web service and turn that data into a pandas
    dataframe for use.
    '''
    
    # initialize:
    
    response = requests.get('https://python.zach.lol/api/v1/stores')
    data = response.json()
    df = pd.DataFrame(data['payload']['stores'])
    
    
    if data['payload']['next_page'] == None:
        return df
    else:
        for x in range(0, data['payload']['max_page']):
            response = requests.get(base_url + data['payload']['next_page'])
            data = response.json()
            df = pd.concat([df, pd.DataFrame(data['payload']['stores'])], ignore_index = True)
        return df
    df = df.reset_index()
    return df

In [ ]:
# Sales function:
# Thanks to Ryvyn and Corey for help!

def get_sales(base_url):
    
    response = requests.get('https://python.zach.lol/api/v1/sales')
    data = response.json()
    data.keys()
    print('max_page: %s' % data['payload']['max_page'])
    print('next_page: %s' % data['payload']['next_page'])
    
    df_sales = pd.DataFrame(data['payload']['sales'])
    
    while data['payload']['next_page'] != "None":
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        print('max_page: %s' % data['payload']['max_page'])
        print('next_page: %s' % data['payload']['next_page'])
        
        
        df_sales = pd.concat([df_sales, pd.DataFrame(data['payload']['sales'])])
        
        if data['payload']['next_page'] == None:
            break
            
    df_sales = df_sales.reset_index()
    print('full_shape', df_sales.shape)
    return df_sales
    

In [ ]:
# Getting power data function:

def get_germany_power(power_url):
    df = pd.DataFrame()
    df = pd.read_csv(power_url, ',')
    
    # now the cleaning:
    df.rename(columns = {"Date": 'date', "Consumption": "consumption", "Wind": "wind", "Solar": "solar", "Wind+Solar": "wind_solar"}, inplace = True)
    
    return df 